# All-Time Scoring Leaders 
Bar Chart Race

## Dependencies

In [1]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
from modules.datasets import BReferenceScraper
import time

## Web Scraping Data
### Wikipedia All-Time List

In [2]:
def all_time_leaders(url: str, output_path=None) -> list:
    html = urlopen(url)
    bs = BeautifulSoup(html, 'lxml')
    
    headers = [
        th.get_text() 
        for th in bs.find_all('tr', limit=2)[0].find_all('th')
        [1:]
    ]

    rows = bs.find_all('tr')[1:]
    rows_data = [
        [td.get_text() for td in rows[i].find_all('td')]
        for i in range(len(rows))
    ][4:54]
    names = [row[1] for row in rows_data]
    names_replaced = [name
                      .replace('*', '') 
                      .replace('\n', '')
                      .replace('^', '')
                      .replace('§', '')
                      .replace('†', '')
                      for name in names]    
    return names_replaced

In [3]:
names = all_time_leaders("https://en.wikipedia.org/wiki/List_of_National_Basketball_Association_career_scoring_leaders")
names

['Kareem Abdul-Jabbar',
 'LeBron James',
 'Karl Malone',
 'Kobe Bryant',
 'Michael Jordan',
 'Dirk Nowitzki',
 'Wilt Chamberlain',
 "Shaquille O'Neal",
 'Carmelo Anthony',
 'Moses Malone',
 'Elvin Hayes',
 'Hakeem Olajuwon',
 'Oscar Robertson',
 'Kevin Durant',
 'Dominique Wilkins',
 'Tim Duncan',
 'Paul Pierce',
 'John Havlicek',
 'Kevin Garnett',
 'Vince Carter',
 'Alex English',
 'Reggie Miller',
 'Jerry West',
 'Patrick Ewing',
 'Ray Allen',
 'Allen Iverson',
 'James Harden',
 'Russell Westbrook',
 'Charles Barkley',
 'Robert Parish',
 'Adrian Dantley',
 'Dwyane Wade',
 'Elgin Baylor',
 'Clyde Drexler',
 'Gary Payton',
 'Larry Bird',
 'Hal Greer',
 'Chris Paul',
 'DeMar DeRozan',
 'Stephen Curry',
 'Walt Bellamy',
 'Pau Gasol',
 'Bob Pettit',
 'David Robinson',
 'George Gervin',
 'LaMarcus Aldridge',
 'Mitch Richmond',
 'Joe Johnson',
 'Tom Chambers',
 'Antawn Jamison']

## Basketball Reference

In [4]:
def breference_url(name: str) -> str:
    names = name.replace('-', ' ').replace("'", '').lower().split(' ')
    url = 'https://www.basketball-reference.com/players/' + names[1][0] + '/' + names[1][0:5] + names[0][0:2] + '01.html'
    return url

def scrap_seasons(name: str) -> pd.DataFrame:
    time.sleep(5)
    url = breference_url(name)
    html = urlopen(url)
    bs = BeautifulSoup(html, 'lxml')
    
    table = bs.find_all('table')[0]
    df = pd.read_html(str(table))[0]
    df = df[~df.Age.isna()]
    df['Name'] = name
    return df

def scrap_seasons_all(names: list) -> pd.DataFrame:
    df = pd.DataFrame()
    for name in names:
        print(name)
        df = pd.concat([df, scrap_seasons(name)])
    df.to_csv('../data/breference/raw/all_time_leaders1.csv', index=False)
    return df

df = scrap_seasons_all(names[0:10])
df

Kareem Abdul-Jabbar


HTTPError: HTTP Error 429: Too Many Requests